# Функция для расчета retention rate

#### Загрузим данные

In [2]:
import pandas as pd

reg_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-ev-varlamov/shared/problem1-reg_data.csv', sep = ';')
auth_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-ev-varlamov/shared/problem1-auth_data.csv', sep = ';')

**Проверяем наличие пропусков и дубликатов**

In [7]:
reg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   reg_ts  1000000 non-null  int64
 1   uid     1000000 non-null  int64
dtypes: int64(2)
memory usage: 15.3 MB


In [8]:
reg_data.describe()

,reg_ts,uid
count,1.000000e+06,1.000000e+06
mean,1.548332e+09,5.552354e+05
std,5.253653e+07,3.206011e+05
min,9.113822e+08,1.000000e+00
25%,1.528034e+09,2.776438e+05
50%,1.564454e+09,5.550455e+05
75%,1.585759e+09,8.329712e+05
max,1.600874e+09,1.110622e+06


In [9]:
reg_data.isnull().sum()

reg_ts    0
uid       0
dtype: int64

In [10]:
reg_data.nunique()

reg_ts    1000000
uid       1000000
dtype: int64

In [11]:
auth_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
dtypes: int64(2)
memory usage: 146.5 MB


In [12]:
auth_data.describe()

,auth_ts,uid
count,9.601013e+06,9.601013e+06
mean,1.548778e+09,3.347198e+05
std,5.160527e+07,2.832669e+05
min,9.113822e+08,1.000000e+00
25%,1.528375e+09,9.599700e+04
50%,1.564577e+09,2.579210e+05
75%,1.585823e+09,5.190250e+05
max,1.600874e+09,1.110622e+06


In [14]:
auth_data.isnull().sum()

auth_ts    0
uid        0
dtype: int64

In [16]:
auth_data.nunique()

auth_ts    9180915
uid        1000000
dtype: int64

#### Преобразуем временные метки в формат даты

In [18]:
reg_data['reg_date'] = pd.to_datetime(reg_data['reg_ts'], unit='s').dt.date
auth_data['auth_date'] = pd.to_datetime(auth_data['auth_ts'], unit='s').dt.date

In [19]:
reg_data

,reg_ts,uid,reg_date
0,911382223,1,1998-11-18
1,932683089,2,1999-07-22
2,947802447,3,2000-01-13
3,959523541,4,2000-05-28
4,969103313,5,2000-09-16
...,...,...,...
999995,1600874034,1110618,2020-09-23
999996,1600874086,1110619,2020-09-23
999997,1600874139,1110620,2020-09-23
999998,1600874191,1110621,2020-09-23


In [20]:
auth_data

,auth_ts,uid,auth_date
0,911382223,1,1998-11-18
1,932683089,2,1999-07-22
2,932921206,2,1999-07-25
3,933393015,2,1999-07-31
4,933875379,2,1999-08-05
...,...,...,...
9601008,1600874034,1110618,2020-09-23
9601009,1600874086,1110619,2020-09-23
9601010,1600874139,1110620,2020-09-23
9601011,1600874191,1110621,2020-09-23


#### Объединяем данные о регистрациях и заходах по uid

In [21]:
data = pd.merge(reg_data, auth_data, on='uid', how='outer', suffixes=('_auth', '_reg'))
data

,reg_ts,uid,reg_date,auth_ts,auth_date
0,911382223,1,1998-11-18,911382223,1998-11-18
1,932683089,2,1999-07-22,932683089,1999-07-22
2,932683089,2,1999-07-22,932921206,1999-07-25
3,932683089,2,1999-07-22,933393015,1999-07-31
4,932683089,2,1999-07-22,933875379,1999-08-05
...,...,...,...,...,...
9601008,1600874034,1110618,2020-09-23,1600874034,2020-09-23
9601009,1600874086,1110619,2020-09-23,1600874086,2020-09-23
9601010,1600874139,1110620,2020-09-23,1600874139,2020-09-23
9601011,1600874191,1110621,2020-09-23,1600874191,2020-09-23


#### Вычисляем количество дней между регистрацией и заходами

In [22]:
data['days_since_reg'] = (data['auth_date'] - data['reg_date']).dt.days
data

,reg_ts,uid,reg_date,auth_ts,auth_date,days_since_reg
0,911382223,1,1998-11-18,911382223,1998-11-18,0
1,932683089,2,1999-07-22,932683089,1999-07-22,0
2,932683089,2,1999-07-22,932921206,1999-07-25,3
3,932683089,2,1999-07-22,933393015,1999-07-31,9
4,932683089,2,1999-07-22,933875379,1999-08-05,14
...,...,...,...,...,...,...
9601008,1600874034,1110618,2020-09-23,1600874034,2020-09-23,0
9601009,1600874086,1110619,2020-09-23,1600874086,2020-09-23,0
9601010,1600874139,1110620,2020-09-23,1600874139,2020-09-23,0
9601011,1600874191,1110621,2020-09-23,1600874191,2020-09-23,0


#### Указываем диапазон дней для расчета retention

In [23]:
day_range = 30

#### Фильтруем данные по дню

In [24]:
data = data[data['days_since_reg'].between(0, day_range)]
data

,reg_ts,uid,reg_date,auth_ts,auth_date,days_since_reg
0,911382223,1,1998-11-18,911382223,1998-11-18,0
1,932683089,2,1999-07-22,932683089,1999-07-22,0
2,932683089,2,1999-07-22,932921206,1999-07-25,3
3,932683089,2,1999-07-22,933393015,1999-07-31,9
4,932683089,2,1999-07-22,933875379,1999-08-05,14
...,...,...,...,...,...,...
9601008,1600874034,1110618,2020-09-23,1600874034,2020-09-23,0
9601009,1600874086,1110619,2020-09-23,1600874086,2020-09-23,0
9601010,1600874139,1110620,2020-09-23,1600874139,2020-09-23,0
9601011,1600874191,1110621,2020-09-23,1600874191,2020-09-23,0


#### Считаем количество зарегистрированных пользователей по дате

In [25]:
reg_counts = reg_data['reg_date'].value_counts().sort_index()
reg_counts

1998-11-18       1
1999-07-22       1
2000-01-13       1
2000-05-28       1
2000-09-16       1
              ... 
2020-09-19    1634
2020-09-20    1636
2020-09-21    1638
2020-09-22    1641
2020-09-23    1048
Name: reg_date, Length: 5110, dtype: int64

#### Считаем количество вернувшихся пользователей на каждый день от даты регистрации

In [26]:
retention_counts = data.groupby(['reg_date', 'days_since_reg'])['uid'].nunique().reset_index()
retention_counts

,reg_date,days_since_reg,uid
0,1998-11-18,0,1
1,1999-07-22,0,1
2,1999-07-22,3,1
3,1999-07-22,9,1
4,1999-07-22,14,1
...,...,...,...
92444,2020-09-21,1,31
92445,2020-09-21,2,49
92446,2020-09-22,0,1641
92447,2020-09-22,1,14


#### Вычисляем retention как долю вернувшихся пользователей

In [27]:
retention_counts = retention_counts.pivot(index='reg_date', columns='days_since_reg', values='uid').fillna(0)
retention = retention_counts.div(reg_counts, axis=0)
retention_counts

days_since_reg,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
reg_date,,,,,,,,,,,,,,,,,,,,,
1998-11-18,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999-07-22,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2000-01-13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-05-28,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-09-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-19,1634.0,30.0,58.0,77.0,57.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-20,1636.0,40.0,71.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-21,1638.0,31.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
retention

days_since_reg,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
reg_date,,,,,,,,,,,,,,,,,,,,,
1998-11-18,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999-07-22,1.0,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2000-01-13,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-05-28,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-09-16,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-19,1.0,0.018360,0.035496,0.047124,0.034884,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-20,1.0,0.024450,0.043399,0.023839,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-21,1.0,0.018926,0.029915,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### В виде функции

In [36]:
def calculate_retention(reg_data, auth_data, start_date=None, end_date=None):
    """
    Вычисляет retention игроков по дням от даты регистрации.

    Args:
        reg_data (pd.DataFrame): Датафрейм с данными о регистрации игроков.
        auth_data (pd.DataFrame): Датафрейм с данными об авторизациях игроков.
        start_date (str, optional): Начальная дата для подсчета retention. 
                                    Формат даты: 'YYYY-MM-DD'. Defaults to None.
        end_date (str, optional): Конечная дата для подсчета retention.
                                  Формат даты: 'YYYY-MM-DD'. Defaults to None.

    Returns:
        pd.DataFrame: Датафрейм с retention rate по дням от даты регистрации.
    """

    # Преобразуем временные метки в формат даты
    reg_data['reg_date'] = pd.to_datetime(reg_data['reg_ts'], unit='s').dt.date
    auth_data['auth_date'] = pd.to_datetime(auth_data['auth_ts'], unit='s').dt.date

    # Фильтруем данные по датам, если указаны
    if start_date is not None:
        reg_data = reg_data[reg_data['reg_date'] >= pd.to_datetime(start_date).date()]
        auth_data = auth_data[auth_data['auth_date'] >= pd.to_datetime(start_date).date()]
    if end_date is not None:
        reg_data = reg_data[reg_data['reg_date'] <= pd.to_datetime(end_date).date()]
        auth_data = auth_data[auth_data['auth_date'] <= pd.to_datetime(end_date).date()]

    # Объединяем данные о регистрациях и заходах по uid
    data = pd.merge(reg_data, auth_data, on='uid', how='outer', suffixes=('_reg', '_auth'))

    # Вычисляем количество дней между регистрацией и заходами
    data['days_since_reg'] = (data['auth_date'] - data['reg_date']).dt.days

    # Считаем количество зарегистрированных пользователей по дате
    reg_counts = reg_data['reg_date'].value_counts().sort_index()

    # Считаем количество вернувшихся пользователей на каждый день от даты регистрации
    retention_counts = data.groupby(['reg_date', 'days_since_reg'])['uid'].nunique().reset_index()

    # Вычисляем retention как долю вернувшихся пользователей
    retention_counts = retention_counts.pivot(index='reg_date', columns='days_since_reg', values='uid').fillna(0)
    retention = retention_counts.div(reg_counts, axis=0)

    return retention

#### Пример использования функции

In [35]:
reg_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-ev-varlamov/shared/problem1-reg_data.csv', sep = ';')
auth_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-ev-varlamov/shared/problem1-auth_data.csv', sep = ';')
retention = calculate_retention(reg_data, auth_data, start_date='2020-09-01', end_date='2020-09-23')
retention

days_since_reg,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0
reg_date,,,,,,,,,,,,,,,,,,,,,
2020-09-01,1.0,0.020177,0.040984,0.042245,0.046028,0.064313,0.056116,0.064313,0.040984,0.054224,...,0.054855,0.042875,0.044136,0.043506,0.050441,0.041614,0.043506,0.035939,0.043506,0.015763
2020-09-02,1.0,0.025189,0.039043,0.046599,0.056675,0.062343,0.071159,0.062972,0.048489,0.040932,...,0.043451,0.049748,0.052897,0.041562,0.048489,0.039043,0.037154,0.046599,0.026448,0.000000
2020-09-03,1.0,0.023256,0.054054,0.047140,0.065368,0.069767,0.085481,0.063482,0.054683,0.062854,...,0.059711,0.054683,0.053426,0.043997,0.048397,0.046512,0.054683,0.023884,0.000000,0.000000
2020-09-04,1.0,0.020088,0.036409,0.036409,0.051475,0.064030,0.072819,0.049592,0.053358,0.043315,...,0.055869,0.043942,0.045198,0.045198,0.042687,0.047709,0.026365,0.000000,0.000000,0.000000
2020-09-05,1.0,0.027569,0.039474,0.046366,0.054511,0.058897,0.074561,0.048872,0.050752,0.054511,...,0.053885,0.045113,0.051378,0.044486,0.047619,0.025063,0.000000,0.000000,0.000000,0.000000
2020-09-06,1.0,0.031270,0.043152,0.054409,0.056285,0.058787,0.068168,0.060663,0.058161,0.055034,...,0.051907,0.055034,0.045654,0.048155,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000
2020-09-07,1.0,0.029357,0.042473,0.053716,0.050593,0.063710,0.073079,0.052467,0.041224,0.056215,...,0.050593,0.042473,0.050593,0.033729,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-09-08,1.0,0.017456,0.039277,0.039277,0.048628,0.076060,0.075436,0.066085,0.051746,0.051746,...,0.051746,0.047382,0.030549,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-09-09,1.0,0.017424,0.032358,0.046671,0.057250,0.066584,0.057250,0.059116,0.043559,0.050404,...,0.048538,0.024269,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
